In [ ]:
pip install pytorch-tabnet torch pandas scikit-learn joblib gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload your `dataset.csv` here


Saving dataset.csv to dataset.csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
import joblib
import os
import shutil

# Load dataset
df = pd.read_csv("dataset.csv")
X = df.drop(columns=["target"])
y = df["target"]

# 5-fold Stratified CV
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"\n🔁 Fold {fold+1}")

    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    # Save the first scaler
    if fold == 0:
        joblib.dump(scaler, "scaler.pkl")

    # Train TabNet
    model = TabNetClassifier(
        n_d=32, n_a=32, n_steps=5, gamma=1.5,
        lambda_sparse=1e-4,
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=1e-2),
        mask_type='entmax'
    )

    model.fit(
        X_train=X_train_scaled, y_train=y_train.values,
        eval_set=[(X_val_scaled, y_val.values)],
        eval_name=["val"],
        eval_metric=["accuracy"],
        max_epochs=200,
        patience=20,
        batch_size=256,
        virtual_batch_size=128
    )

    preds = model.predict(X_val_scaled)
    acc = accuracy_score(y_val, preds)
    print(f"✅ Fold {fold+1} Accuracy: {acc*100:.2f}%")
    accuracies.append(acc)

# Save final model
model.save_model("model")

print(f"\n🎯 Average CV Accuracy: {np.mean(accuracies)*100:.2f}%")



🔁 Fold 1


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.65828 | val_accuracy: 0.63025 |  0:00:00s
epoch 1  | loss: 0.71259 | val_accuracy: 0.71008 |  0:00:00s
epoch 2  | loss: 0.57734 | val_accuracy: 0.77731 |  0:00:00s
epoch 3  | loss: 0.52947 | val_accuracy: 0.76891 |  0:00:01s
epoch 4  | loss: 0.45456 | val_accuracy: 0.76471 |  0:00:01s
epoch 5  | loss: 0.4711  | val_accuracy: 0.7437  |  0:00:02s
epoch 6  | loss: 0.38889 | val_accuracy: 0.80252 |  0:00:03s
epoch 7  | loss: 0.35212 | val_accuracy: 0.83193 |  0:00:04s
epoch 8  | loss: 0.37636 | val_accuracy: 0.79412 |  0:00:05s
epoch 9  | loss: 0.3992  | val_accuracy: 0.78151 |  0:00:05s
epoch 10 | loss: 0.35586 | val_accuracy: 0.80252 |  0:00:06s
epoch 11 | loss: 0.34322 | val_accuracy: 0.84034 |  0:00:06s
epoch 12 | loss: 0.32818 | val_accuracy: 0.83193 |  0:00:07s
epoch 13 | loss: 0.31786 | val_accuracy: 0.83613 |  0:00:07s
epoch 14 | loss: 0.29629 | val_accuracy: 0.86555 |  0:00:07s
epoch 15 | loss: 0.3326  | val_accuracy: 0.84454 |  0:00:08s
epoch 16 | loss: 0.28149

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


✅ Fold 1 Accuracy: 86.55%

🔁 Fold 2


...

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.74598 | val_accuracy: 0.65966 |  0:00:00s
epoch 1  | loss: 0.71075 | val_accuracy: 0.7395  |  0:00:00s
epoch 2  | loss: 0.59113 | val_accuracy: 0.73109 |  0:00:00s
epoch 3  | loss: 0.54975 | val_accuracy: 0.7563  |  0:00:00s
epoch 4  | loss: 0.47554 | val_accuracy: 0.7521  |  0:00:00s
epoch 5  | loss: 0.41568 | val_accuracy: 0.77311 |  0:00:00s
epoch 6  | loss: 0.41119 | val_accuracy: 0.76471 |  0:00:00s
epoch 7  | loss: 0.4333  | val_accuracy: 0.79832 |  0:00:01s
epoch 8  | loss: 0.37787 | val_accuracy: 0.83193 |  0:00:01s
epoch 9  | loss: 0.36078 | val_accuracy: 0.81933 |  0:00:01s
epoch 10 | loss: 0.32669 | val_accuracy: 0.82353 |  0:00:01s
epoch 11 | loss: 0.34938 | val_accuracy: 0.81933 |  0:00:01s
epoch 12 | loss: 0.32386 | val_accuracy: 0.80672 |  0:00:01s
epoch 13 | loss: 0.32601 | val_accuracy: 0.81092 |  0:00:01s
epoch 14 | loss: 0.30524 | val_accuracy: 0.83193 |  0:00:02s
epoch 15 | loss: 0.3614  | val_accuracy: 0.84874 |  0:00:02s
epoch 16 | loss: 0.30211

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.76212 | val_accuracy: 0.60924 |  0:00:00s
epoch 1  | loss: 0.74533 | val_accuracy: 0.7437  |  0:00:00s
epoch 2  | loss: 0.59617 | val_accuracy: 0.7437  |  0:00:00s
epoch 3  | loss: 0.57505 | val_accuracy: 0.7605  |  0:00:00s
epoch 4  | loss: 0.46414 | val_accuracy: 0.78992 |  0:00:00s
epoch 5  | loss: 0.45873 | val_accuracy: 0.77311 |  0:00:00s
epoch 6  | loss: 0.40967 | val_accuracy: 0.78151 |  0:00:00s
epoch 7  | loss: 0.38789 | val_accuracy: 0.81933 |  0:00:01s
epoch 8  | loss: 0.34806 | val_accuracy: 0.78992 |  0:00:01s
epoch 9  | loss: 0.35221 | val_accuracy: 0.81513 |  0:00:01s
epoch 10 | loss: 0.37395 | val_accuracy: 0.84034 |  0:00:01s
epoch 11 | loss: 0.36888 | val_accuracy: 0.83193 |  0:00:01s
epoch 12 | loss: 0.32466 | val_accuracy: 0.84034 |  0:00:01s
epoch 13 | loss: 0.29909 | val_accuracy: 0.86134 |  0:00:01s
epoch 14 | loss: 0.29885 | val_accuracy: 0.85714 |  0:00:02s
epoch 15 | loss: 0.30406 | val_accuracy: 0.85714 |  0:00:02s
epoch 16 | loss: 0.30562

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [ ]:
import gradio as gr
import numpy as np
import joblib
from pytorch_tabnet.tab_model import TabNetClassifier

# Load model and scaler
model = TabNetClassifier()
model.load_model("model.zip")
scaler = joblib.load("scaler.pkl")

# Prediction function
def predict(age, sex, cp, trestbps, chol, fbs, restecg,
            thalach, exang, oldpeak, slope):
    data = np.array([[age, sex, cp, trestbps, chol, fbs, restecg,
                      thalach, exang, oldpeak, slope]])
    scaled = scaler.transform(data)
    pred = model.predict(scaled)
    return "🫀 Heart Disease Detected" if pred[0] == 1 else "✅ Normal"

# Labels & Tooltips
sex_labels = {0: "Female", 1: "Male"}
cp_labels = {
    1: "Typical Angina",
    2: "Atypical Angina",
    3: "Non-anginal Pain",
    4: "Asymptomatic"
}
restecg_labels = {
    0: "Normal",
    1: "ST-T Abnormality",
    2: "Left Ventricular Hypertrophy"
}
slope_labels = {
    1: "Upsloping",
    2: "Flat",
    3: "Downsloping"
}

# Interface
with gr.Blocks(theme=gr.themes.Soft(), title="Heart Disease Predictor") as demo:
    gr.Markdown("# 🫀 Heart Disease Prediction App")
    gr.Markdown("Get a fast medical prediction based on vitals and test results using a deep learning model trained on heart disease data.")

    with gr.Row():
        with gr.Column():
            age = gr.Number(label="Age", info="Age of the patient in years", value=50)
            sex = gr.Radio(choices=list(sex_labels.keys()), label="Sex", value=1, info="0 = Female, 1 = Male")
            cp = gr.Radio(choices=list(cp_labels.keys()), label="Chest Pain Type", value=4, info=str(cp_labels))
            trestbps = gr.Number(label="Resting Blood Pressure (mm Hg)", value=130)
            chol = gr.Number(label="Serum Cholesterol (mg/dL)", value=250)
            fbs = gr.Radio(choices=[0, 1], label="Fasting Blood Sugar > 120 mg/dL?", value=0)

        with gr.Column():
            restecg = gr.Radio(choices=list(restecg_labels.keys()), label="Resting ECG Results", value=1, info=str(restecg_labels))
            thalach = gr.Number(label="Maximum Heart Rate Achieved", value=150)
            exang = gr.Radio(choices=[0, 1], label="Exercise Induced Angina?", value=0)
            oldpeak = gr.Number(label="ST Depression Induced by Exercise", value=1.5)
            slope = gr.Radio(choices=list(slope_labels.keys()), label="Slope of Peak Exercise ST Segment", value=2, info=str(slope_labels))

    submit_btn = gr.Button("🔍 Predict", variant="primary")

    output = gr.Textbox(label="Diagnosis")

    # On submit
    submit_btn.click(fn=predict,
                     inputs=[age, sex, cp, trestbps, chol, fbs, restecg,
                             thalach, exang, oldpeak, slope],
                     outputs=output)

    gr.Markdown("### 📊 Example Inputs")
    examples = gr.Examples(
        examples=[
            [63, 1, 3, 145, 233, 1, 0, 150, 0, 2.3, 3],
            [45, 0, 2, 120, 200, 0, 1, 160, 0, 1.0, 2],
            [58, 1, 4, 140, 230, 1, 2, 130, 1, 3.2, 1],
            [52, 0, 1, 130, 180, 0, 0, 150, 0, 0.0, 2],
            [69, 1, 4, 160, 289, 1, 2, 110, 1, 2.0, 3]
        ],
        inputs=[age, sex, cp, trestbps, chol, fbs, restecg,
                thalach, exang, oldpeak, slope]
    )

demo.launch()


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0f563b0ffb478fb0c3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import shutil
import os

# Make sure your app.py, model.zip, scaler.pkl, and requirements.txt are in this folder
project_dir = "heart_disease_gradio_app"
os.makedirs(project_dir, exist_ok=True)

# Save app.py
with open(f"{project_dir}/app.py", "w") as f:
    f.write("""PASTE FULL app.py CODE HERE""")

# Save requirements.txt
with open(f"{project_dir}/requirements.txt", "w") as f:
    f.write("gradio\ntorch\npandas\nnumpy\njoblib\nscikit-learn\npytorch-tabnet")

# Copy model and scaler into folder
shutil.copy("model.zip", f"{project_dir}/model.zip")
shutil.copy("scaler.pkl", f"{project_dir}/scaler.pkl")

# Zip it
shutil.make_archive("heart_disease_gradio_app", 'zip', project_dir)

# Download
from google.colab import files
files.download("heart_disease_gradio_app.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Zip it
shutil.make_archive("heart_disease_gradio_app", 'zip', project_dir)

# Download
from google.colab import files
files.download("heart_disease_gradio_app.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>